In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from keras.optimizers import SGD
from keras.utils import np_utils

train = pd.read_csv('./Leaf Dataset/train.csv')
test = pd.read_csv('./Leaf Dataset/test.csv')

def encode(train, test):
    label_encoder = LabelEncoder().fit(train.species)
    labels = label_encoder.transform(train.species)
    classes = list(label_encoder.classes_)

    train = train.drop(['species', 'id'], axis=1)
    test = test.drop('id', axis=1)

    return train, labels, test, classes

train, labels, test, classes = encode(train, test)

# standardize train features
scaler = StandardScaler().fit(train.values)
scaled_train = scaler.transform(train.values)

# split train data into train and validation
sss = StratifiedShuffleSplit(test_size=0.1, random_state=23)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]
    

nb_features = 64 # number of features per features type (shape, texture, margin)   
nb_class = len(classes)

# reshape train data
X_train_r = np.zeros((len(X_train), nb_features, 3))
X_train_r[:, :, 0] = X_train[:, :nb_features]
X_train_r[:, :, 1] = X_train[:, nb_features:128]
X_train_r[:, :, 2] = X_train[:, 128:]

# reshape validation data
X_valid_r = np.zeros((len(X_valid), nb_features, 3))
X_valid_r[:, :, 0] = X_valid[:, :nb_features]
X_valid_r[:, :, 1] = X_valid[:, nb_features:128]
X_valid_r[:, :, 2] = X_valid[:, 128:]

# Keras model with one Convolution1D layer
# unfortunately more number of covnolutional layers, filters and filters lenght 
# don't give better accuracy
model = Sequential()
model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))


y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

nb_epoch = 15
model.fit(X_train_r, y_train, nb_epoch=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)

/home/manas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(64, 3), filters=512, kernel_size=1)`
/home/manas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 891 samples, validate on 99 samples
Epoch 1/15
891/891 [==============================] - 79s 88ms/step - loss: 3.6740 - accuracy: 0.2301 - val_loss: 1.5057 - val_accuracy: 0.6970
Epoch 2/15
891/891 [==============================] - 35s 39ms/step - loss: 0.6754 - accuracy: 0.8418 - val_loss: 0.3479 - val_accuracy: 0.8990
Epoch 3/15
891/891 [==============================] - 33s 37ms/step - loss: 0.1279 - accuracy: 0.9697 - val_loss: 0.3408 - val_accuracy: 0.9091
Epoch 4/15
891/891 [==============================] - 33s 37ms/step - loss: 0.0717 - accuracy: 0.9865 - val_loss: 0.1656 - val_accuracy: 0.9495
Epoch 5/15
688/891 [======================>.......] - ETA: 7s - loss: 0.0369 - accuracy: 0.9942

KeyboardInterrupt: 